In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense,LeakyReLU,Input
from keras.optimizers import Adam

##Initialisation
batch_size = 256
step_per_epoch = 3750
epochs = 20

##Load the dataset
(x_train,x_test),(y_train,y_test) = mnist.load_data()
## Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
x_train = (x_train.astype(np.float32) - 127.5) / 127.5
x_train = x_train.reshape(-1,28*28*1)



##Define standalone generator network 
def Generator():
  generator = Sequential()
  generator.add(Dense(210,input_dim=100))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(510))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(810))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(1010))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(28*28*1,activation="tanh"))

  generator.compile(optimizer=Adam(0.0002,0.5),loss="binary_crossentropy")

  return generator

##Define standalone Discriminator network    
def Descriminator():
  descriminator = Sequential()
  descriminator.add(Dense(1010,input_dim=28*28*1))
  descriminator.add(LeakyReLU(0.2))

  descriminator.add(Dense(810))
  descriminator.add(LeakyReLU(0.2))

  descriminator.add(Dense(510))
  descriminator.add(LeakyReLU(0.2))

  descriminator.add(Dense(210))
  descriminator.add(LeakyReLU(0.2))

  descriminator.add(Dense(1,activation="sigmoid"))
  descriminator.compile(optimizer=Adam(0.0002,0.5),loss="binary_crossentropy")
  return descriminator

## Define the combined generator and discriminator model, for updating the generator 
##This builds the discriminator  
desc = Descriminator()
##This builds the Generator
gen = Generator()
##This ensures that when we combine our networks we only train the Generator.
##While generator training we do not want discriminator weights to be adjusted. 
##This Doesn't affect the above descriminator training.
desc.trainable = False
##Our random input 
gan_input = Input(shape=(100,))

##In a GAN the Generator network takes an input to produce its images
fake_img = gen(gan_input)

##This specifies that Discriminator will take the images generated by our Generator  
gan_output = desc(fake_img)

##The combined model of GAN with  generator and discriminator inputs
gan = Model(gan_input,gan_output)
## compile model of GAN
gan.compile(loss="binary_crossentropy",optimizer=Adam(0.0002,0.5))

## Manually enumerate epoch
for epoch in range(epochs):
## Enumerate batches over the training set  
  for batch in range(step_per_epoch):
##Select a random batch of images
    noise = np.random.normal(0,1,size=(batch_size,100))
## Generate a batch of fake images
    fake = gen.predict(noise)
## Train the discriminator on real and fake images
    real = x_train[np.random.randint(0,x_train.shape[0],size=batch_size)]
    x = np.concatenate((real,fake))
    label_real = np.ones(2*batch_size)
    label_real[:batch_size] = 0.9
##Take average loss from real and fake images for discriminator
    desc_loss = desc.train_on_batch(x,label_real)
##Take average loss from real and fake images for generator 
    label_fake = np.zeros(batch_size)
    gen_loss = gan.train_on_batch(noise,label_fake)

## In order to keep track of our training process, we print the
##progress
  print(f"Epoch : {epoch} / Descriminator Loss : {desc_loss} / Generator Loss : {gen_loss}")


##This function saves our images for us to view
def show_image(noise):
 images = gen.predict(noise)
 plt.figure(figsize=(5,4))
 for i,image in enumerate(images):
  plt.subplot(5,4,i+1)
  plt.imshow(images[i].reshape((28,28)))
 plt.show()
noise = np.random.normal(0,1,size=(20,100))
show_image(noise)


 



11493376/11490434 [==============================] - 0s 0us/step
Epoch : 0 / Descriminator Loss : 0.18195068836212158 / Generator Loss : 4.609687805175781
Epoch : 1 / Descriminator Loss : 0.18475976586341858 / Generator Loss : 4.2197980880737305
Epoch : 2 / Descriminator Loss : 0.18840208649635315 / Generator Loss : 3.851132869720459
